<a href="https://colab.research.google.com/github/hatish2001/Children-Story-generation-using-LLM-s/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
# The following packages are installed by default on Colab
# If you're running on another platform, you may need to install them with
# pip, conda, brew, or other package managers.
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [2]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2024-12-06 21:18:26--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt [following]
--2024-12-06 21:18:26--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  10.5MB/s    in 0.4s    

2024-12-06 21:18:27 (10.5 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [3]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks we discussed in class.

In [4]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.7983852028846741),
 ('prince', 0.7517792582511902),
 ('york', 0.7444249391555786),
 ('warwick', 0.7233059406280518),
 ('duke', 0.7192990779876709),
 ('clarence', 0.7055583000183105),
 ('princess', 0.6948035359382629),
 ('son', 0.6894858479499817),
 ('cardinal', 0.6798964142799377),
 ('percy', 0.6755530834197998)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [5]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9638245701789856),
 ('cleopatra', 0.9333956241607666),
 ('iago', 0.9238488078117371),
 ('mrs', 0.9182373881340027),
 ('ham', 0.9097301959991455),
 ('countess', 0.905004620552063),
 ('cassio', 0.9033157825469971),
 ('imogen', 0.9031842350959778),
 ('emilia', 0.8990395069122314),
 ('jul', 0.8906999826431274)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [6]:
model.wv.similarity('othello', 'desdemona')

0.9638246

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [7]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
    # Compute the dot product of v1 and v2
    dot_product = np.dot(v1, v2)
    # Compute the norms of v1 and v2
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    # Compute the cosine similarity
    return dot_product / (norm_v1 * norm_v2)

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.96382475

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [22]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2024-12-06 21:30:18--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/30k.en.vec [following]
--2024-12-06 21:30:18--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M  39.6MB/s    in 1.6s    

2024-12-06 21:30:20 (39.6 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2024-12-06 21:30:20--  http://www.ccs.neu.edu/home/dasmith/courses/

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [23]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1}

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [14]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):

  mostSimilar = ''  # Initialize with an empty string
  similarity = 0    # Initialize with the minimum similarity
  for word, candidateVec in vecDict.items():
    current_similarity = cosim(vec, candidateVec)
    if current_similarity > similarity:
      mostSimilar = word
      similarity = current_similarity
  return(mostSimilar, similarity)



## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [16]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2024-12-06 21:26:06--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/links.tab [following]
--2024-12-06 21:26:06--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  3.62MB/s    in 0.4s    

2024-12-06 21:26:07 (3.62 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [17]:
links[302]

['academy', 'fr', 'académie']

We construct a test set for English-French from the first 1000 links between those languages.

In [18]:
frtest = [x for x in links if x[1] == "fr"][0:1000]
frtest[0:10]

[['aalborg', 'fr', 'aalborg'],
 ['aarhus', 'fr', 'aarhus'],
 ['aba', 'fr', 'aba'],
 ['abad', 'fr', 'abad'],
 ['abandon', 'fr', 'abandon'],
 ['abbas', 'fr', 'abbas'],
 ['abbasid', 'fr', 'abbassides'],
 ['abbess', 'fr', 'abbesse'],
 ['abbey', 'fr', 'abbaye'],
 ['abbot', 'fr', 'abbé']]

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article in this test set `frtest` is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary.

Since many articles, e.g., about named entities, have the same title in English and French, use the identity function as a baseline and compute its accuracy. In other words, how often would you find the right French articles by simply echoing the English title as if it were French? In the ten example links above, `aalborg` and `aarhus` (two cities in Denmark) are the same in English and French. Remember to iterate only over the 1000 linked Wikipedia articles in the test set, not the entire embedding dictionary.

In [19]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
accuracy = 0
baselineAccuracy = 0

correct_retrievals = 0
baseline_correct = 0
total_test_cases = 0

for link in frtest:
    eng_word = link[0]  # English term
    fr_word = link[2]   # French term

    # Skip if the English word is not in the English embedding dictionary
    if eng_word not in envec:
        continue

    # Retrieve the most similar word in French embeddings for the English term
    retrieved_word, _ = mostSimilar(envec[eng_word], frvec)

    # Check if the retrieved word matches the actual French word
    if retrieved_word == fr_word:
        correct_retrievals += 1

    # Baseline: Check if English and French words are identical
    if eng_word == fr_word:
        baseline_correct += 1

    total_test_cases += 1

# Compute accuracy
if total_test_cases > 0:
    accuracy = correct_retrievals / total_test_cases
    baselineAccuracy = baseline_correct / total_test_cases

# Print results
print(f"Accuracy: {accuracy}")
print(f"Baseline Accuracy: {baselineAccuracy}")

Accuracy: 0.575
Baseline Accuracy: 0.661


**TODO**: Compute the accuracy, i.e. precision@1, of the embeddings and of the identity function for the first 1000 links between English and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [24]:
## TODO: Compute English-X Wikipedia retrieval accuracy.

# Compute English-X Wikipedia retrieval accuracy
def compute_accuracy(test_set, envec, xvec):
    """
    Compute the accuracy (precision@1) for the embedding-based retrieval
    and the identity function for the given test set between English and language X.
    """
    correct_retrievals = 0
    baseline_correct = 0
    total_test_cases = 0

    for link in test_set:
        eng_word = link[0]  # English term
        x_word = link[2]    # X language term

        # Skip if the English word is not in the English embedding dictionary
        if eng_word not in envec:
            continue

        # Retrieve the most similar word in the X embeddings for the English term
        retrieved_word, _ = mostSimilar(envec[eng_word], xvec)

        # Check if the retrieved word matches the actual X word
        if retrieved_word == x_word:
            correct_retrievals += 1

        # Baseline: Check if English and X words are identical
        if eng_word == x_word:
            baseline_correct += 1

        total_test_cases += 1

    # Compute accuracy and baseline accuracy
    accuracy = correct_retrievals / total_test_cases if total_test_cases > 0 else 0
    baseline_accuracy = baseline_correct / total_test_cases if total_test_cases > 0 else 0

    return accuracy, baseline_accuracy

# Example usage with another language (e.g., Spanish 'esvec')
test_set = [x for x in links if x[1] == "es"][:1000]  # Assuming `links` contains English-X pairs
accuracy, baseline_accuracy = compute_accuracy(test_set, envec, esvec)  # Replace `esvec` with the target language embeddings

print(f"Accuracy for English-X retrieval: {accuracy}")
print(f"Baseline accuracy: {baseline_accuracy}")



Accuracy for English-X retrieval: 0.536
Baseline accuracy: 0.518


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [25]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.

import heapq

def recall_at_10_and_mrr(test_set, envec, xvec):
    """
    Compute recall@10 and mean reciprocal rank (MRR)@10 for retrieving the 10 nearest neighbors
    in the embedding space of the target language (xvec).
    """
    recall_at_10 = 0
    mrr_at_10 = 0
    total_test_cases = 0

    for link in test_set:
        eng_word = link[0]  # English term
        x_word = link[2]    # Ground truth word in target language

        # Skip if the English word is not in the English embedding dictionary
        if eng_word not in envec:
            continue

        # Compute similarities to all target language words
        similarities = []
        for target_word, target_vec in xvec.items():
            similarity = cosim(envec[eng_word], target_vec)
            similarities.append((similarity, target_word))

        # Retrieve the top 10 most similar words
        top_10_neighbors = heapq.nlargest(10, similarities, key=lambda x: x[0])

        # Extract the words from the top 10 neighbors
        top_10_words = [neighbor[1] for neighbor in top_10_neighbors]

        # Check if the ground truth word is in the top 10 neighbors
        if x_word in top_10_words:
            recall_at_10 += 1
            # Compute rank (1-based index) of the ground truth word in the top 10
            rank = top_10_words.index(x_word) + 1
            mrr_at_10 += 1 / rank

        total_test_cases += 1

    # Normalize recall@10 and MRR@10
    recall_at_10 = recall_at_10 / total_test_cases if total_test_cases > 0 else 0
    mrr_at_10 = mrr_at_10 / total_test_cases if total_test_cases > 0 else 0

    return recall_at_10, mrr_at_10

# Example usage with a target language (e.g., French 'frvec')
test_set = [x for x in links if x[1] == "fr"][:1000]  # Assuming `links` contains English-French pairs
recall_10, mrr_10 = recall_at_10_and_mrr(test_set, envec, frvec)

print(f"Recall@10: {recall_10}")
print(f"MRR@10: {mrr_10}")


Recall@10: 0.644
MRR@10: 0.6013718253968254


## Speeding up Vector Search (for CS6200)

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.

A relatively easy way to get started with fast vector similarity search is to install Meta's `faiss` (Facebook AI Similarity Search) package and read [the tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started).

In [29]:
# Outside of colab, you may need a different package manager.
# !apt install libomp-dev
!pip install --upgrade faiss-cpu
# Use this line instead if you have a GPU.
# !python -m pip install --upgrade faiss-gpu
import faiss
import time

def build_faiss_index(embedding_dict):
    """
    Build a FAISS index for a given embedding dictionary.
    """
    # Convert embedding dictionary to a numpy array and map words to indices
    vectors = np.array(list(embedding_dict.values())).astype('float32')
    words = list(embedding_dict.keys())

    # Create FAISS index
    index = faiss.IndexFlatL2(vectors.shape[1])  # L2 distance for nearest neighbor search
    index.add(vectors)  # Add vectors to the index

    return index, words

def faiss_search(index, query_vector, top_k=10):
    """
    Perform a search for the top_k nearest neighbors using a FAISS index.
    """
    distances, indices = index.search(query_vector.reshape(1, -1), top_k)
    return indices[0], distances[0]

def evaluate_with_faiss(test_set, envec, xvec, index, x_words, top_k=10):
    """
    Evaluate recall@10 and MRR@10 using the FAISS index.
    """
    recall_at_10 = 0
    mrr_at_10 = 0
    total_test_cases = 0

    for link in test_set:
        eng_word = link[0]
        x_word = link[2]

        # Skip if the English word is not in the English embedding dictionary
        if eng_word not in envec:
            continue

        # Retrieve the FAISS index for the top_k nearest neighbors
        query_vector = np.array(envec[eng_word]).astype('float32')
        indices, _ = faiss_search(index, query_vector, top_k)

        # Map indices back to words
        neighbors = [x_words[idx] for idx in indices]

        # Compute recall@10
        if x_word in neighbors:
            recall_at_10 += 1
            rank = neighbors.index(x_word) + 1
            mrr_at_10 += 1 / rank

        total_test_cases += 1

    recall_at_10 = recall_at_10 / total_test_cases if total_test_cases > 0 else 0
    mrr_at_10 = mrr_at_10 / total_test_cases if total_test_cases > 0 else 0
    return recall_at_10, mrr_at_10

# Brute-force search for comparison
def brute_force_search(query_vector, target_dict, top_k=10):
    """
    Perform a brute-force search to find the top_k nearest neighbors.
    """
    similarities = []
    for word, target_vec in target_dict.items():
        similarity = cosim(query_vector, target_vec)
        similarities.append((similarity, word))
    # Sort by similarity and take the top_k
    top_k_neighbors = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_k]
    return [neighbor[1] for neighbor in top_k_neighbors]

# Evaluate Brute Force
def evaluate_brute_force(test_set, envec, xvec, top_k=10):
    recall_at_10 = 0
    mrr_at_10 = 0
    total_test_cases = 0

    for link in test_set:
        eng_word = link[0]
        x_word = link[2]

        # Skip if the English word is not in the embedding dictionary
        if eng_word not in envec:
            continue

        # Perform brute force search
        query_vector = np.array(envec[eng_word])
        neighbors = brute_force_search(query_vector, xvec, top_k)

        # Compute recall@10
        if x_word in neighbors:
            recall_at_10 += 1
            rank = neighbors.index(x_word) + 1
            mrr_at_10 += 1 / rank

        total_test_cases += 1

    recall_at_10 = recall_at_10 / total_test_cases if total_test_cases > 0 else 0
    mrr_at_10 = mrr_at_10 / total_test_cases if total_test_cases > 0 else 0
    return recall_at_10, mrr_at_10

# Compare Brute Force and FAISS
def compare_brute_force_faiss(test_set, envec, frvec, fr_index, fr_words):
    print("Evaluating Brute Force...")
    start_time = time.time()
    brute_recall_10, brute_mrr_10 = evaluate_brute_force(test_set, envec, frvec)
    brute_time = time.time() - start_time
    print(f"Brute Force - Recall@10: {brute_recall_10}, MRR@10: {brute_mrr_10}, Time: {brute_time} seconds")

    print("\nEvaluating FAISS...")
    start_time = time.time()
    faiss_recall_10, faiss_mrr_10 = evaluate_with_faiss(test_set, envec, frvec, fr_index, fr_words)
    faiss_time = time.time() - start_time
    print(f"FAISS - Recall@10: {faiss_recall_10}, MRR@10: {faiss_mrr_10}, Time: {faiss_time} seconds")

    print("\nComparison:")
    print(f"Recall@10 - Brute Force: {brute_recall_10}, FAISS: {faiss_recall_10}")
    print(f"MRR@10 - Brute Force: {brute_mrr_10}, FAISS: {faiss_mrr_10}")
    print(f"Time - Brute Force: {brute_time} seconds, FAISS: {faiss_time} seconds")

# Example usage for comparison
compare_brute_force_faiss(frtest, envec, frvec, fr_index, fr_words)

Evaluating Brute Force...
Brute Force - Recall@10: 0.644, MRR@10: 0.6013718253968254, Time: 264.80918884277344 seconds

Evaluating FAISS...
FAISS - Recall@10: 0.644, MRR@10: 0.6013718253968254, Time: 3.0005006790161133 seconds

Comparison:
Recall@10 - Brute Force: 0.644, FAISS: 0.644
MRR@10 - Brute Force: 0.6013718253968254, FAISS: 0.6013718253968254
Time - Brute Force: 264.80918884277344 seconds, FAISS: 3.0005006790161133 seconds


**TODO**: Create two vector indexes, for the FastText embeddings of French and for the other language you evaluated above. Use `faiss` to find the 10 nearest neighbors for the top 1000 Wikipedia headwords you evaluated for English-French and the English-X as above.

First, measure the _effectiveness_ of this approximate vector search approach. How does the R@10 and MRR@10 using `faiss` compare to the brute-force search you did above?

Second, measure the _efficiency_ of this approach. How long in seconds does finding nearest neighbors for 1000 headwords by brute force compare to using `faiss`? (For this exercise, don't worry about amortizing indexing costs.)